In [32]:
from hlsf.models import *
import matplotlib.pyplot as plt
import numpy as np
from lmfit.models import MoffatModel
%matplotlib qt

In [33]:
config = 'H'
lamp = 'Kr'
nb_line = 177
lamps = ['Ar', 'Kr', 'Ne', 'Xe']

In [34]:
lsf_data = LSF_DATA(f"../exposures/ARC-{lamp}_CLEAR_20MAS_{config}_PRM.fits", f"../text/{lamp}.txt", f"../exposures/WAVECAL_TABLE_20MAS_{config}.fits", 
                        f"../exposures/SLITLET_TABLE_20MAS_{config}.fits")
lsf_data.get_line_list().keys()

dict_keys([171, 172, 173, 174, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 192, 193, 194, 195, 196, 197, 198, 199])

In [35]:
data = lsf_data.get_data_line(nb_line)
waves = data['map_wave']
w_0 = data['waveline']
intensity = data['intensity']

In [36]:
mod = MoffatModel()
ind = np.argmin(abs(max(intensity) - intensity))
A = max(intensity)
mu = waves[ind] - w_0
ind_half = np.argmin(abs(max(intensity)/2 - intensity))
sigma = abs(waves[ind_half] - w_0 - mu)
init_params = mod.make_params(amplitude=A, center=mu, sigma=sigma, beta=1)
init_params

name,value,initial value,min,max,vary,expression
amplitude,0.40202525,None,-inf,inf,True,
center,-0.02102233,None,-inf,inf,True,
sigma,1.15238184,None,0.00000000,inf,True,
beta,1.00000000,None,-inf,inf,True,
fwhm,2.30476367,None,-inf,inf,False,"2*sigma*sqrt(2**(1.0/max(1e-3, beta))-1)"
height,0.40202525,None,-inf,inf,False,amplitude


In [37]:
waves_linspace = np.linspace(min(waves-w_0), max(waves-w_0), len(waves))
eval_intensity = mod.eval(init_params, x=waves_linspace)
fig = plt.figure()
ax = plt.axes()
lsf_data.plot_line(nb_line, ax)
ax.plot(waves_linspace, eval_intensity, color='red')
plt.show()

In [38]:
out = mod.fit(intensity, init_params, x=waves-w_0)

In [39]:
out.params

name,value,standard error,relative error,initial value,min,max,vary,expression
amplitude,0.40149383,2.8780e-04,(0.07%),0.4020252525806427,-inf,inf,True,
center,4.9084e-04,7.3594e-04,(149.93%),-0.02102232955803629,-inf,inf,True,
sigma,37.6899858,46.3352450,(122.94%),1.1523818354216928,0.00000000,inf,True,
beta,719.933460,1768.30051,(245.62%),1,-inf,inf,True,
fwhm,2.33952162,0.00237050,(0.10%),2.3047636708433856,-inf,inf,False,"2*sigma*sqrt(2**(1.0/max(1e-3, beta))-1)"
height,0.40149383,2.8780e-04,(0.07%),0.4020252525806427,-inf,inf,False,amplitude


In [40]:
waves_linspace = np.linspace(min(waves-w_0), max(waves-w_0), len(waves))
eval_intensity = mod.eval(out.params, x=waves_linspace)
fig = plt.figure()
ax = plt.axes()
lsf_data.plot_line(nb_line, ax)
ax.plot(waves_linspace, eval_intensity, color='red')
plt.show()